
 # <font color='blue'>Chatbot - prompt engineering - LangChain</font>

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.10.12


In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark.
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.8 MB/s eta 0:00:00


In [3]:
!pip install -q transformers einops accelerate langchain bitsandbytes xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 922.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [4]:
# Imports
import torch
import langchain
import transformers
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline, PromptTemplate, LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.prompts import ChatPromptTemplate
from IPython.display import display, Markdown

In [5]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

torch       : 2.1.2
langchain   : 0.1.0
transformers: 4.35.2



In [6]:
# Checando a GPU
!nvidia-smi

Sat Jan 13 10:47:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Definindo o LLM Para IA Generativa

Este Lab usa o modelo de linguagem **gpt2-small-portuguese** do HuggingFace para gerar respostas aprimoradas com base em uma sequência de texto inicial.

O principal objetivo é melhorar as respostas geradas pelo modelo **gpt2-small-portuguese** usando técnicas como Prompt Templates e Sequential Chains.

O **gpt2-small-portuguese ** é um modelo de linguagem de inteligência artificial baseado no modelo GPT2. É uma versão em português deste modelo que foi treinada para gerar texto em espanhol. .

https://huggingface.co/pierreguillou/gpt2-small-portuguese

In [32]:
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch

tokenizer = AutoTokenizer.from_pretrained("pierreguillou/gpt2-small-portuguese")
model = AutoModelWithLMHead.from_pretrained("pierreguillou/gpt2-small-portuguese")

# Get sequence length max of 1024
tokenizer.model_max_length=1024

model.eval()  # disable dropout (or leave in train mode to finetune)


tokenizer_config.json:   0%|          | 0.00/92.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/850k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/508k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [7]:
# Repositório do modelo
#modelo = "recogna-nlp/bode-7b-alpaca-pt-br"

In [8]:
# Tokenizador
#tokenizer = AutoTokenizer.from_pretrained(modelo)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: recogna-nlp/bode-7b-alpaca-pt-br does not appear to have a file named config.json. Checkout 'https://huggingface.co/recogna-nlp/bode-7b-alpaca-pt-br/main' for available files.

In [33]:
# Cria o pipeline
pipeline = pipeline("text-generation",

                    # Modelo que será usado para geração de texto
                    model = model,

                    # Tokenizer
                    tokenizer = tokenizer,

                    # Tipo dos dados para computação
                    torch_dtype = torch.bfloat16,

                    # Se devemos confiar no código remoto ao carregar o modelo.
                    # Isso só deve ser definido como True se você confiar na origem do modelo.
                    trust_remote_code = True,

                    # O dispositivo a ser usado para os cálculos do modelo.
                    device_map = "auto",

                    # O comprimento máximo do texto a ser gerado.
                    max_length = 512,

                    # Se deve usar a amostragem ao gerar texto.
                    do_sample = True,

                    # O número de candidatos a manter durante a amostragem.
                    top_k = 2,

                    # O número de sequências de texto a serem geradas.
                    num_return_sequences = 1,

                    # O ID do token de fim de sequência.
                    eos_token_id = tokenizer.eos_token_id)

A temperatura é um hiperparâmetro em muitos modelos de linguagem que controla a aleatoriedade da saída do modelo. Um valor de temperatura mais baixo (como 0.2) faz com que o modelo produza saídas mais determinísticas, enquanto um valor de temperatura mais alto faz com que o modelo produza saídas mais aleatórias.

In [34]:
# HuggingFacePipeline
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {"temperature": 0.2})

### Testando o Modelo Sem o Uso de Templates (Direto com Prompts)

In [35]:
def imprime_resposta(string):
    display(Markdown(string))

In [36]:
imprime_resposta(llm("Descreva as belezas do Brasil"))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


, e o Brasil é o país mais bonito do mundo.

O Brasil é um país que possui um grande número de universidades, institutos e institutos de pesquisa, e é o país com o maior número de universidades do mundo.

O Brasil é um país com uma população de cerca de habitantes.

A população brasileira de acordo com o censo de 2010 era de habitantes.

A população brasileira é formada por:



A língua oficial é o inglês.

O Brasil é um país de língua oficial.

O Brasil é o país que possui a maior porcentagem de pessoas alfabetizadas.

O Brasil é o país que possui o maior número de alfabetizados.

O Brasil é o país que possui o maior percentual de analfabetos.

O Brasil é o país com a maior percentagem de pessoas com deficiência intelectual.

A população brasileira é formada por:


























































A população brasileira é formada por:




























A população brasileira é formada por:
















































A população brasileira é formada por:

















































































































































A população brasileira de acordo com o censo de 2010 era de habitantes.





In [37]:
imprime_resposta(llm("Computadores são o futuro?"))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.




A ideia de uma computação paralela surgiu em meados do século XIX. A ideia de uma computação paralela surgiu em meados do século XIX, quando o cientista alemão Ernst Macht, em sua tese sobre a teoria dos números, propôs a ideia de que a computação paralela seria possível.

Em meados do século XIX, a computação paralela foi desenvolvida por John von Neumann, que foi o primeiro cientista a propor a ideia de que a computação paralela seria possível.

Em meados do século XX, a computação paralela foi desenvolvida por John von Neumann, que foi o primeiro cientista a propor a idéia de que a computação paralela seria possível.

Em meados do século XXI, a computação paralela foi desenvolvida por John von Neumann, que foi o primeiro cientista a propor a ideia de que a computação paralela seria possível.

A computação paralela é um conceito de computação paralela que se refere a um conjunto de técnicas que são usadas para resolver problemas de computação.

A computação paralela pode ser definida como uma técnica que é usada para resolver problemas de computação em uma máquina.

A computação paralela pode ser definida como uma técnica que é usada para resolver problemas de computação em um computador.

A computação paralela pode ser definida como uma técnica que é usada para resolver problemas de computação em um computador.

A computação paralela pode ser definida como uma técnica que é usada para resolver problemas de computação em um computador.

Uma computação paralela pode ser definida como um conjunto de técnicas que são usadas para resolver problemas de computação em um computador.

Uma computação paralela pode ser definida como uma técnica que é usada para resolver problemas de computação em um computador.

Uma computação paralela pode ser definida como uma técnica que é usada para resolver problemas de computação em um computador.

A computação paralela pode ser definida como uma técnica que é usada para resolver problemas de computação em um computador.

Uma computação paralela pode ser definida como uma técnica que é usada para resolver problemas de computação em um computador.

A computação paralela pode ser definida como uma técnica que é usada para resolver problemas de computação em um computador.

A computação paralela pode ser definida como uma técnica que é usada para resolver problemas de computação em um computador.

Uma computação paralela pode ser definida como uma técnica que é usada para resolver problemas de computação em um computador.

Uma computação paralela pode ser definida como uma técnica que é usado para resolver problemas de computação em um computador.

A computação paralela é um conceito de computação paralela que se refere a um

In [38]:
imprime_resposta(llm("O que é a inteligencia artificial?"))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


 A inteligencia é o conhecimento que se dá por meio de um processo que se realiza com o objetivo de obter a verdade. A inteligencia é a capacidade de se conhecer e compreender o mundo, a verdade, o mundo e a realidade.

O conceito de inteligencia é uma das mais antigas teorias de inteligencia, que foram desenvolvidas por Carl Jung e Sigmund Freud. A ideia é que a mente humana é composta de uma série de partes, cada uma com uma função específica, que são capazes de se comunicar e se relacionar.

A ideia de inteligencia é que o cérebro é formado por uma estrutura que é capaz de se comunicar com outras partes da estrutura, e que a mente humana é composta de uma estrutura que é capaz de se relacionar com outras partes da estrutura, e que a mente humana é capaz de se relacionar com outras partes da estrutura.

A ideia de inteligencia é que o conhecimento é o resultado de um processo de aprendizagem que se realiza com a intenção de obter a verdade. A inteligencia é a capacidade de se conhecer e compreender o mundo, a verdade, a verdade, a realidade e o mundo. A ideia de inteligencia é que a mente é formada por uma estrutura que é capaz de se comunicar com outras partes da estrutura, e que a mente humana é capaz de se relacionar com outras partes da estrutura, e que a mente humana é capaz de se relacionar com outras partes da estrutura, e que a mente humana é capaz de se relacionar com outras partes da estrutura, e que a mente humana é capaz de se relacionar com outras partes da estrutura.

A teoria de Carl Jung, que foi desenvolvida por Carl Jung e Sigmund Freud, é a mais abrangente e abrangente, e é a mais abrangente e abrangente teoria de inteligencia, e a teoria mais abrangente e abrangente de toda a teoria de Carl Jung.

A teoria de Carl Jung, desenvolvida por Carl Jung e Sigmund Freud, é a mais abrangente e abrangente, e é a mais abrangente e abrangente teoria de inteligencia, e a teoria mais abrangente de toda a teoria de Sigmund Freud.

A teoria de Carl Jung, desenvolvida por Carl Jung e Sigmund Freud, é a mais abrangente e abrangente, e é a mais abrangente e abrangente teoria de toda a teoria de Sigmund Freud.

A teoria de Carl Jung, desenvolvida por Carl Jung e Sigmund Freud, é a mais abrangente e abrangente, e é a mais abrangente e abrangente teoria de toda a teoria de Sigmund Freud.

A teoria de Carl Jung

### Testando o Modelo com Prompt Templates do LangChain

In [39]:
#langchain.debug = True

> Exemplo 1

In [40]:
# Template
template = """
  Eres un chatbot experto en psicologia y neurociencia.
  Responde a la siguiente pregunta con respuestas brillantes.
  Sólo tienes que escribir la respuesta, no más.
  Pregunta: {pregunta}
  Respuesta:
"""

In [41]:
# Prompt
prompt = PromptTemplate(template = template, input_variables = ["pregunta"])
print(prompt)

input_variables=['pregunta'] template='\n  Eres un chatbot experto en psicologia y neurociencia.\n  Responde a la siguiente pregunta con respuestas brillantes.\n  Sólo tienes que escribir la respuesta, no más.\n  Pregunta: {pregunta}\n  Respuesta:\n'


In [42]:
# Chain
llm_chain = LLMChain(prompt = prompt, llm = llm)

In [43]:
# Questão
question = "Explique o que é atenção?"

In [44]:
# Executa a chain
exec_exemplo1 = llm_chain.run(question)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [45]:
# Print
imprime_resposta(exec_exemplo1)

  Atenção, atenção, atenção, atenção, atenção.

 Atenção, atenção, atenção, atenção.

 Atenção, atenção, atenção, atenção.

 Atenção, atenção, atenção, atenção.

 Atenção, atenção, atenção, atenção, atenção.

 Atenção, atenção, atenção, atenção, atenção.

 Atenção, atenção, atenção, atenção,

 Atenção, atenção, atenção, atenção,

 Atenção, atenção, atenção, atenção,



     Atenção, atenção, atenção,

           


             

                

              

             

             

             

            

            

            

           
A Igreja de São João Batista de Jesus Cristo dos Santos dos Últimos Dias, em Jerusalém, foi uma igreja de Roma, fundada em 1838, e que foi a primeira igreja de Jesus Cristo dos Santos dos Últimos Dias. A igreja foi construída em um terreno de propriedade de um proprietário de terras de Roma, que era o proprietário da igreja, e que foi o primeiro a se estabelecer em Jerusalém, e que foi a primeira igreja a se estabelecer em Jerusalém. A igreja foi construída em uma área de propriedade de um homem, que era o proprietário da igreja. A igreja foi construída em um terreno de propriedade de um homem e que foi o primeiro a se estabelecer em Jerusalém. A igreja foi construída em uma área de propriedade de

> Exemplo 2

In [46]:
# Template
template = """
  Eres un chatbot inteligente, experto en Python, conoces todo sobre el lenguaje.
  Solo tienes responder a la pregunta no más.
  Pregunta: {pregunta}
  Respuesta:
"""

In [47]:
# Prompt
prompt = PromptTemplate(template = template, input_variables = ["pregunta"])
print(prompt)

input_variables=['pregunta'] template='\n  Eres un chatbot inteligente, experto en Python, conoces todo sobre el lenguaje.\n  Solo tienes responder a la pregunta no más.\n  Pregunta: {pregunta}\n  Respuesta:\n'


In [48]:
# Chain
llm_chain = LLMChain(prompt = prompt, llm = llm)

In [49]:
# Questão
question = "Que es un diccionario?"

In [50]:
# Executa a chain
exec_exemplo2 = llm_chain.run(question)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [51]:
imprime_resposta(exec_exemplo2)

  Eres, es un diccionario?

Eres, es un diccionario?

Eres, es un diccionario?

Eres, es un diccionario?

Eres, es un diccionario?
Eres, es un diccionario?
Eres, es un diccionario?
Eres, es un diccionario? 
Eres, es un diccionario?
Eres, es un diccionario?
Eres, es un diccionario? 
Eres, es un diccionario?
Eres, es un diccionario? 
Eres, es un diccionario?
Eres, es un diccionario? 
Eres, es un diccionario?
Eres, es un diccionario?
Eres, es un diccionario?
Eres

Eres
Eres
Eres
Eres
Eres
Eres
Eres
Eres
Eres
Eres
Eres
Eres
Eres
Eres
Eres  Eres, en el lenguaje, en el la pampa, en el la pampa, en la pampa, en el la pampa, en la pampa, en la pampa, en el la pampa, en la pampa, en el la pampa, en el la pampa, en el la pampa, en el la pampa, en el la pampa, en el la pampa, en el la pampa, en el la pampa, en el la pampa, en el la pampa, en el la pampa, en el la pampa, en el la pampa, en el la pampa, en el la pampa, en el la pampa, en el la pampa, en el la pampa, en el la pampa, en el la pampa, en el la pampa, en el la pampa, en el la pampa, en el la pampa, en el la pampa, en el la pampa, en el la pampa

In [52]:
# Questão
question = "Cuales son los tipos de datos basicos en python?"

In [53]:
imprime_resposta(llm_chain.run(question))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


 
Acho que el lenguaje não é uma língua de linguagem natural, mas uma língua de aprendizado, que se pode dizer com certeza que a linguagem natural não é uma língua natural, mas uma linguagem de aprendizado.

Acho que a linguagem natural não é uma língua natural, e sim uma língua de aprendizado.

Acho que el lenguaje não é uma língua natural.

Acho que a linguagem natural não é uma língua de aprendizado.

Acho que el lenguaje não é uma língua natural.

Acho que a linguagem natural não é uma língua de aprendizado.

Acho que el lenguaje não é uma língua natural.

Acho que a linguagem natural não é uma língua de aprendizado.

Acho que a linguagem natural não é uma língua de aprendizado.

O programa de computador é uma linguagem de aprendizado.

A linguagem de aprendizado é uma linguagem de programação, que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programação que é uma linguagem de programa de computador.

O termo linguagem de programação é uma palavra de origem inglesa que significa "uma linguagem de programação que é uma linguagem de programação que

## Sequential Chains

In [54]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "pregunta: Cual es el mejor nombre para describir una empresa que fabrica {producto}, solo dame un nombre inovador e interesante y solo responde con el nombre. respuesta:"
)

In [55]:
# Chain 1
chain_one = LLMChain(llm = llm, prompt = first_prompt)

In [56]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Escribe una descripcion creativa de la siguiente compañia: {company_name}"
)

In [57]:
# chain 2
chain_two = LLMChain(llm = llm, prompt = second_prompt)

In [58]:
# SimpleSequentialChain
overall_simple_chain = SimpleSequentialChain(chains = [chain_one, chain_two], verbose = True)

In [59]:
product = "Burritos veganos"

In [60]:
exec_chain = overall_simple_chain.run(product)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.




> Entering new SimpleSequentialChain chain...


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


 el mejor nombre para describir una empresa que fabrica Burritos veganos, solo dame un nombre inovador e interesante y solo responde con el nombre.

O primeiro produto da empresa foi o Burrito de Burrito, que foi vendido em um mercado interno de aproximadamente US$ 2,5 milhões. O Burrito de Burrito de Burrito de Burrito de Burrito de Burrito foi lançado em outubro de 2005.

Em 2006, o Burrito de Burrito de Burrito de Burrito de Burrito foi lançado em um mercado interno de aproximadamente US$ 1,5 milhões. O produto foi lançado em outubro de 2006, e foi lançado em outubro de 2007.

Em 2008, o Burrito de Burrito de Burrito de Burrito de Burrito foi lançado em um mercado interno de aproximadamente US$ 1,5 milhões. O produto foi lançado em novembro de 2008, e foi lançado em novembro de 2009.

Em 2010, a Burrito de Burrito de Burrito de Burrito de Burrito de Burrito de Burrito foi lançado em um mercado interno de aproximadamente US$ 1,5 milhões.

Em 2011, a Burrito de Burrito de Burrito de B

In [61]:
imprime_resposta(exec_chain)

rito de Burrito de Burrito de Burrito de Burrito de Burrito de Burrito de Burrito de Burrito de Burrito

# Fim